# Yonsei Major parser


# PyMuPDF4LLM

In [25]:
# !pip install "pymupdf>=1.24.0" pymupdf4llm

In [26]:
import pymupdf4llm

pdf_path = "data/yonsei/raw/yonsei_raw.pdf"

# page_chunks=True -> 페이지별로 dict 반환
pages = pymupdf4llm.to_markdown(
    pdf_path,
    page_chunks=True,
)

print(len(pages))        # 전체 페이지 수
print(pages[0].keys())   # dict 키 확인: text, tables, images, ...

Consider using the pymupdf_layout package for a greatly improved page layout analysis.
1180
dict_keys(['metadata', 'toc_items', 'tables', 'images', 'graphics', 'text', 'words'])


In [27]:
from typing import List, Tuple

def split_markdown_text_and_tables(md: str) -> Tuple[str, List[str]]:
    """
    전체 Markdown 문자열에서
    - 일반 텍스트(표 제외)
    - 표(마크다운 테이블) 목록
    을 분리해서 반환.
    """
    lines = md.splitlines()
    tables: List[str] = []
    text_lines: List[str] = []

    in_table = False
    cur_table: List[str] = []

    def flush_table():
        nonlocal cur_table
        if cur_table:
            tables.append("\n".join(cur_table).strip())
            cur_table = []

    for line in lines:
        stripped = line.strip()
        is_table_line = stripped.startswith("|") and "|" in stripped

        if is_table_line:
            # 테이블 블록 진입 또는 연속
            in_table = True
            cur_table.append(line)
        else:
            # 테이블 블록이 끝났다면 tables에 저장
            if in_table:
                flush_table()
                in_table = False
            # 일반 텍스트로 저장
            text_lines.append(line)

    # 파일 끝에서 테이블로 끝났을 수도 있으니 한 번 더 flush
    if in_table:
        flush_table()

    text_md = "\n".join(text_lines).strip()
    return text_md, tables


In [28]:
page_text_md = {}
page_tables_md = {}

for i, page in enumerate(pages):
    page_no = i + 1  # 1-based 페이지 번호

    text_md, tables_md = split_markdown_text_and_tables(page["text"])

    page_text_md[page_no] = text_md
    page_tables_md[page_no] = tables_md

# 예: 133페이지 텍스트/표 확인
print("=== [133 페이지 텍스트] ===")
print(page_text_md.get(133, "no text"))

print("\n=== [133 페이지 테이블들] ===")
for idx, t in enumerate(page_tables_md.get(133, []), start=1):
    print(f"\n[Table {idx}]\n{t}\n")

=== [133 페이지 텍스트] ===
진리와 자유로 인류의 가능성을 열어가는 연세
Trust, Pathfinding, Dedication


해당학과에서 지정하는 총 36~39학점을 추가로 이수하여야 한다.
(, 단 두 전공영역에서 공통으로 인정되는 전공과목의 학점은 두 전공 영역에서 각기 전공학점으로 인정한다. 전공학점의
중복인정은 이과대학 개 전공과 생명시스템대학 개 전공시스템생물학생화학에 한하며전공하는 학과에서 중복인정의 6 2 (, ),
범위와 학점 수에 제한을 둘 경우 해당학과의 기준에 따른다.)


3. 이과대학공통 전공기초 이수 요건



















  - 학과별로 전공기초 이수요건을 정한 학과는 해당 전공기초 필수과목을 이수하여야 한다.


￭ 이과대학공통 전공기초 교과목 ￭


MAT1001 미분적분학과벡터해석(1) Calculus and Vector Analysis (1)
실수의 집합함수와 함수의 극한도함수함수의 연속성미분법미분의 응용적분적분법적분의 응용극좌표무한 급수와,,,,,,,,,,
수렴성함수의 전개 등 일변수 함수의 미분 적분에 관한 여러 가지 기본 개념과 응용성을 이해한다,


MAT1002 미분적분학과벡터해석(2) Calculus and Vector Analysis (2)
선형벡터 공간행렬식편도함수중적분중적분의 응용벡터의 미분다변수 함수선적분면적분,,,,,,,,, Green의 정리, Stokes의
정리, Gauss 정리 등을 이해한다.


MAT1016 미분적분학과벡터해석심화( )(1) Calculus and Vector Analysis (Advanced) (1)
일반반과 비슷한 주제를 다루나 수학적으로 엄밀한 접근법을 사용하며 특히 실수 체계의 정의와 극한의 개념간의 연관성을
강조하고 등장하는 모든 정리를 증명하며대수수론실 해석 및 복소 해석학의 수학적 기반을 마련한다,,, .

MAT1017 미분적분학과벡터해석심화( )(2) Calculus and Vector Analysis (Advanced) (2)
일반반

In [29]:
# 이미 어딘가에서:
# pages = pymupdf4llm.to_markdown(pdf_path, page_chunks=True)
import os

start_page = 133
end_page = 282
output_dir = "data/yonsei/md_pages"
os.makedirs(output_dir, exist_ok=True)

for page_no in range(start_page, end_page + 1):
    index = page_no - 1
    if index < 0 or index >= len(pages):
        continue

    md_text = pages[index].get("text", "").strip()
    if not md_text:
        continue

    out_path = os.path.join(output_dir, f"page_{page_no}.md")
    with open(out_path, "w", encoding="utf-8") as f:
        f.write(md_text)

### Markdown 에서 표만 추출

In [30]:
from typing import List

def extract_tables_from_markdown(md: str) -> List[str]:
    """
    전체 Markdown 문자열에서
    - 마크다운 테이블(`| ... |`) 블록만 리스트로 추출
    """
    lines = md.splitlines()
    tables: List[str] = []

    in_table = False
    cur_table: List[str] = []

    def flush_table():
        nonlocal cur_table
        if cur_table:
            # 양 끝 공백 제거
            tables.append("\n".join(cur_table).strip())
            cur_table = []

    for line in lines:
        stripped = line.strip()
        # 간단한 테이블 판별: '|' 로 시작하고, 최소 1개 이상 '|' 포함
        is_table_line = stripped.startswith("|") and "|" in stripped

        if is_table_line:
            in_table = True
            cur_table.append(line)
        else:
            if in_table:
                # 테이블 블록 종료
                flush_table()
                in_table = False

    # 파일 끝이 테이블로 끝났을 수도 있으니 마무리
    if in_table:
        flush_table()

    return tables

In [31]:
import os
from pathlib import Path

input_dir = Path("data/yonsei/md_pages")    # page_133.md 등 있는 폴더
output_dir = Path("data/yonsei/md_tables")  # 표만 모을 폴더

output_dir.mkdir(parents=True, exist_ok=True)

start_page = 133
end_page = 282

all_tables_merged = []  # 전체 표를 한 파일에 모으고 싶을 때 사용

for page_no in range(start_page, end_page + 1):
    md_path = input_dir / f"page_{page_no}.md"
    if not md_path.exists():
        print(f"{md_path} 가 없습니다. 건너뜀.")
        continue

    md_text = md_path.read_text(encoding="utf-8")
    tables = extract_tables_from_markdown(md_text)

    if not tables:
        print(f"페이지 {page_no}: 테이블 없음")
        continue

    print(f"✅ 페이지 {page_no}: {len(tables)}개 테이블 추출")

    # 1) 페이지·테이블 단위로 별도 md 파일 저장
    for idx, tbl_md in enumerate(tables, start=1):
        out_path = output_dir / f"page_{page_no}_table_{idx}.md"
        out_path.write_text(tbl_md + "\n", encoding="utf-8")

        # 2) 전체 합본 파일용으로도 쌓기
        all_tables_merged.append(f"## Page {page_no} - Table {idx}\n\n{tbl_md}\n")

# 3) 전체 표를 한 md 파일로 합치기 (선택)
merged_path = output_dir / "all_tables.md"
merged_path.write_text("\n\n".join(all_tables_merged), encoding="utf-8")

print(f"\n🎉 전체 표 합본: {merged_path}")

✅ 페이지 133: 1개 테이블 추출
페이지 134: 테이블 없음
✅ 페이지 135: 1개 테이블 추출
✅ 페이지 136: 2개 테이블 추출
✅ 페이지 137: 1개 테이블 추출
페이지 138: 테이블 없음
페이지 139: 테이블 없음
페이지 140: 테이블 없음
페이지 141: 테이블 없음
페이지 142: 테이블 없음
✅ 페이지 143: 2개 테이블 추출
✅ 페이지 144: 1개 테이블 추출
페이지 145: 테이블 없음
페이지 146: 테이블 없음
✅ 페이지 147: 1개 테이블 추출
✅ 페이지 148: 1개 테이블 추출
✅ 페이지 149: 1개 테이블 추출
페이지 150: 테이블 없음
페이지 151: 테이블 없음
✅ 페이지 152: 1개 테이블 추출
✅ 페이지 153: 2개 테이블 추출
✅ 페이지 154: 1개 테이블 추출
페이지 155: 테이블 없음
페이지 156: 테이블 없음
페이지 157: 테이블 없음
페이지 158: 테이블 없음
✅ 페이지 159: 1개 테이블 추출
✅ 페이지 160: 2개 테이블 추출
페이지 161: 테이블 없음
페이지 162: 테이블 없음
✅ 페이지 163: 2개 테이블 추출
✅ 페이지 164: 2개 테이블 추출
✅ 페이지 165: 1개 테이블 추출
페이지 166: 테이블 없음
페이지 167: 테이블 없음
페이지 168: 테이블 없음
페이지 169: 테이블 없음
✅ 페이지 170: 1개 테이블 추출
✅ 페이지 171: 3개 테이블 추출
페이지 172: 테이블 없음
페이지 173: 테이블 없음
페이지 174: 테이블 없음
✅ 페이지 175: 2개 테이블 추출
✅ 페이지 176: 2개 테이블 추출
✅ 페이지 177: 1개 테이블 추출
페이지 178: 테이블 없음
페이지 179: 테이블 없음
페이지 180: 테이블 없음
✅ 페이지 181: 2개 테이블 추출
✅ 페이지 182: 2개 테이블 추출
✅ 페이지 183: 2개 테이블 추출
✅ 페이지 184: 1개 테이블 추출
페이지 185: 테이블 없음
페이지 186: 테이블 없음
페이지 18

---

### Json 파싱

In [35]:
from pathlib import Path

md_path = Path("data/yonsei/md_pages/page_133.md")  # 또는 data/yonsei/md_pages/page_133.md
md_text = md_path.read_text(encoding="utf-8")

courses = parse_courses_from_markdown(md_text)

# 결과 예시 확인
for c in courses:
    print(c["subject_number"])
    print(c["name"])
    print(c["description"][:120], "...")
    print("-" * 40)

MAT1001
미분적분학과벡터해석(1)
실수의 집합함수와 함수의 극한도함수함수의 연속성미분법미분의 응용적분적분법적분의 응용극좌표무한 급수와,,,,,,,,,, 수렴성함수의 전개 등 일변수 함수의 미분 적분에 관한 여러 가지 기본 개념과 응용성을 이해한다, ...
----------------------------------------
MAT1002
미분적분학과벡터해석(2)
선형벡터 공간행렬식편도함수중적분중적분의 응용벡터의 미분다변수 함수선적분면적분,,,,,,,,, Green의 정리, Stokes의 정리, Gauss 정리 등을 이해한다. ...
----------------------------------------
MAT1016
미분적분학과벡터해석심화( )(1)
일반반과 비슷한 주제를 다루나 수학적으로 엄밀한 접근법을 사용하며 특히 실수 체계의 정의와 극한의 개념간의 연관성을 강조하고 등장하는 모든 정리를 증명하며대수수론실 해석 및 복소 해석학의 수학적 기반을 마련한다,,, ...
----------------------------------------
MAT1017
미분적분학과벡터해석심화( )(2)
일반반과 비슷한 주제를 다루나 수학적으로 엄밀한 접근법을 사용하며 특히 선형대수의 기초 개념을 이용하여 임의 차원의 유클리드 공간 상 미분적분과 스톡스 정리를 다루고기하 및 위상수학 이해의 기반을 마련한다, . ...
----------------------------------------
MAT2001
고등미적분학(1)
일계및고계 상미분방정식의 이론및 수치해진동운동연립선형미분방정식에 관한 초기값문제역학계,,,, Laplace변환과 응용, Bessel방정식, Legendre방정식벡터 와 벡터공간벡터함수의 미분,, ...
----------------------------------------
MAT2002
고등미적분학(2)
모델링, 파동방정식, 열전도 방정식의 경계값 문제와 Fourier급수, 복소변수와 응용 (Cauchy Riemann방정식, 복소평면위의선적분등각

In [36]:
base_dir = Path("data/yonsei/md_pages")
start_page, end_page = 133, 282

all_courses = []

for page in range(start_page, end_page + 1):
    md_file = base_dir / f"page_{page}.md"
    if not md_file.exists():
        continue
    md_text = md_file.read_text(encoding="utf-8")

    page_courses = parse_courses_from_markdown(md_text)
    # 필요하면 page 정보도 붙이기
    for c in page_courses:
        c["page"] = page

    all_courses.extend(page_courses)

with open("yonsei_이과대학_공통전공기초.json", "w", encoding="utf-8") as f:
    json.dump(all_courses, f, ensure_ascii=False, indent=2)

저장 완료: yonsei_science_common.json


In [38]:
from pathlib import Path
import json

def normalize_folder_name(name: str) -> str:
    parts = name.split("_", 1)
    return parts[1] if len(parts) == 2 else name

def build_department_courses(department_dir: Path) -> list[dict[str, str]]:
    courses_by_key: dict[tuple[str, str], dict[str, str]] = {}

    for md_path in sorted(department_dir.glob("*.md")):
        md_text = md_path.read_text(encoding="utf-8")
        for course in parse_courses_from_markdown(md_text):
            code = course.get("subject_number", "").strip()
            category = course.get("category") or "?? ???"
            key = (category, code)
            if not code or key in courses_by_key:
                continue

            courses_by_key[key] = {
                "grade_semester": "",
                "course_classification": "",
                "????": code,
                "name": course.get("name", ""),
                "description": course.get("description", ""),
                "category": category,
            }

    ordered_keys = sorted(courses_by_key.keys())
    return [courses_by_key[key] for key in ordered_keys]

root_dir = Path("data/yonsei/md_pages")
yonsei_courses = {"???": {}}

for college_dir in sorted(root_dir.iterdir()):
    if not college_dir.is_dir():
        continue

    college_name = normalize_folder_name(college_dir.name)
    college_payload = {}

    for department_dir in sorted(college_dir.iterdir()):
        if not department_dir.is_dir():
            continue

        department_name = normalize_folder_name(department_dir.name)
        college_payload[department_name] = build_department_courses(department_dir)

    yonsei_courses["???"][college_name] = college_payload

output_json = Path("yonsei_courses.json")
output_json.write_text(
    json.dumps(yonsei_courses, ensure_ascii=False, indent=2),
    encoding="utf-8",
)

print(f"?? ??: {output_json}")
print(f"???? ?: {len(yonsei_courses['???'])}")


생성 완료: yonsei_courses.json
단과대학 수: 4


In [ ]:
import fitz

doc = fitz.open("data/yonsei/raw/yonsei_raw.pdf")
page = doc[132]  # 예시
raw_text = page.get_text("text")  # 또는 "blocks"
# 여기서 기존에 짠 정규식/파서 돌려서 JSON 생성

'진리와 자유로 인류의 가능성을 열어가는 연세\nTrust, Pathfinding, Dedication\n132  \n연세대학교 대학요람\n2025 \n■ \n \n해당학과에서 지정하는 총 \n학점을 추가로 이수하여야 한다\n36~39\n.  \n단\n두 전공영역에서 공통으로 인정되는 전공과목의 학점은 두 전공 영역에서 각기 전공학점으로 인정한다\n전공학점의 \n   ( , \n. \n중복인정은 이과대학 개 전공과 생명시스템대학 개 전공시스템생물학생화학에 한하며전공하는 학과에서 중복인정의 \n6\n2\n(\n, \n)\n, \n범위와 학점 수에 제한을 둘 경우 해당학과의 기준에 따른다.)\n이과대학공통 전공기초 이수 요건\n3. \n학과별로 전공기초 이수요건을 정한 학과는 해당 전공기초 필수과목을 이수하여야 한다\n   * \n.  \n이과대학공통 전공기초 교과목 \n￭\n￭\n미분적분학과벡터해석\nMAT1001 \n(1) Calculus and Vector Analysis (1)\n실수의 집합함수와 함수의 극한도함수함수의 연속성미분법미분의 응용적분적분법적분의 응용극좌표무한 급수와 \n, \n, \n, \n, \n, \n, \n, \n, \n, \n, \n수렴성함수의 전개 등 일변수 함수의 미분 적분에 관한 여러 가지 기본 개념과 응용성을 이해한다\n, \n미분적분학과벡터해석\nMAT1002 \n(2) Calculus and Vector Analysis (2)\n선형벡터 공간행렬식편도함수중적분중적분의 응용벡터의 미분다변수 함수선적분면적분\n의 정리\n의 \n, \n, \n, \n, \n, \n, \n, \n, \n, Green\n, Stokes\n정리\n정리 등을 이해한다\n, Gauss \n.\nMAT1016 미분적분학과벡터해석심화\n(\n)(1) Calculus and Vector Analysis (Advanced) (1)\n일반반과 비슷한 주제를 다루나 수학적으로 엄밀한 접근법을 사용하며 특히 실수 체계의 정의와 극한의 개념간의 연관성을 \n강조하고 등장